In [230]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [231]:
data = pd.read_csv('clean_data.csv')
df = data.copy()
df = df.drop(['Unnamed: 0', 'Date de publication', 'lieu'], axis=1)
df['Type de contrat'].fillna('cdi', inplace=True)
df

,Intitulé du poste,competences,Nom de la société,Type de contrat,salaire_minimum,salaire_maximum
0,data analyst,"support, sql, data, management, business, obje...",fed it,cdd,45000.0,50000.0
1,data analyst,"data, management, business, intelligence, base...",fed it,cdi,60000.0,62000.0
2,data scientist,"marketing, data, management, support",fed it,cdi,60000.0,65000.0
3,consultant,"big, data, oracle, cloud, sap, python",selescope,cdi,60000.0,70000.0
4,chef de projet,"marketing, outils, digital, ux, mobile",hays france,cdi,60000.0,70000.0
5,data scientist,"si, data, management, data, quality, méthodolo...",michael page,cdi,55000.0,60000.0
6,business analyst,"business, intelligence, crm, intelligence, art...",selescope,cdi,70000.0,100000.0
7,business analyst,"intelligence, artificielle, business, intellig...",selescope,cdi,70000.0,100000.0
8,business analyst,"business, intelligence, marketing, intelligenc...",fed it,cdi,35000.0,45000.0
9,data scientist,"machine, learning, méthodologie, agile",key consulting,cdi,45000.0,70000.0


In [232]:
# transformation de la colonne compétences
df['competences'] = df['competences'].fillna('agile, sql, support, python, bootstrap')
cv = CountVectorizer() 
trans_comp = cv.fit_transform(df['competences']).toarray()
trans_comp
# 
# trans_comp = cv.fit_transform(df['competences'])
# trans_comp

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [233]:
ohe = OneHotEncoder()
X_cat = df.select_dtypes(include=[object])
X_cat = X_cat.drop('competences', axis=1)
X_cat = ohe.fit_transform(X_cat).toarray()
X_cat

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,

In [235]:
y = df['salaire_maximum']
X = pd.concat([pd.DataFrame(X_cat), pd.DataFrame(trans_comp)])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

line_reg = LinearRegression()
line_reg.fit(X_train, y_train)
prediction = line_reg.predict(X_test)

linear_score = r2_score(y_test, prediction)
print("LR :", round(linear_score, 5))

LR : 0.26142
